## Kaitlyn Perkovic - Spotify API Data Report
### October 11th, 2024
##### The hypothesis I want to test is between two songs from two different albums from my favorite artist: glaive. My hypothesis is that "god is dead" from "a bit of a mad one" will have a lower loudness value than "Nobodys Fault / Accept My Own" from "May It Never Falter." After giving these songs a listen, I think that "Nobodys Fault / Accept My Own" sounds louder. Using the data provided from Spotify' API, I will be able to tell exactly which song is louder.
##### "The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db." source: https://developer.spotify.com/documentation/web-api/reference/get-audio-features
###### Note: the variable I will be using for "a bit of a mad one" is "aboamo" and "minf" for "May It Never Falter."

In [1]:
import urllib
import requests
import pandas as pd
import json
import base64

Import calls for the software and files I will need to complete this report. urllib allows to read urls. requests allows to make a request to a website and API's. pandas, abbreviated to pd, allows for easier data organization such as tables. json allows to read and write in json format for the websites and API's that use json. base64 allows to use the encode and decode to encode/decode credentials.

In [2]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID + ":" + SessionKey).encode("ascii"))
    header = {'Authorization': 'Basic {}'.format(encoded_key.decode("ascii"))}
    response = requests.post(url, data = data, headers = header)
    print(response.status_code)
    return response.json()['access_token']

def defines the function get_session_token with the variables SessionID and SessionKey that will be called from Spotify's API. url = ... defines the url that will request the SessionID and SessionKey. data = ... defines what type of credentials I will need- I am not super sure on this. encoded_key = ... combines SessionID and SessionKey into a string using (str(SessionID + ":" + SessionKey). header = includes the SessionID and SessionKey combined string. response = ... sends a request to Spotify with url=, data= and headers=. print allows to print the status code (200's are what I want). return allows for a response of the access token in json format

I am not sure what "ascii", 'Authorization' or 'Basic {}' do. 

In [3]:
keys = pd.read_csv('keys.txt')

keys = calls for pandas to read the keys.txt file located in jupyter notebook.

In [4]:
#keys

Calls for the function to read the file.

In [5]:
access_token = get_session_token(keys['Client_ID'].iloc[0], keys['Client_Secret'].iloc[0])

200


Defines the access_token function to retrieve the session token keys located in the keys.txt file. keys selects the 0th element (using .iloc[0]) in "Client_ID" and the 0th element in "Client_Secret."

In [6]:
#access_token

Calls for the function to be processed.

In [7]:
t_features_ep = 'https://api.spotify.com/v1/audio-features'
track_ep = 'https://api.spotify.com/v1/tracks'
ab_tracks_ep = 'https://api.spotify.com/v1/albums/{}/tracks'

These functions call for the Spotify API's that I will need (audio features, tracks, and albums)- the ID of the album will go in {}.

In [8]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

def defines the api_call function with the endpoint_url and api_header variables. response = ... sends a request to get the endpoint url and to the api header. print will print the status code. return will return the response in a json format.

In [9]:
session_header = {'Authorization': 'Bearer {}'.format(access_token)}

session_header is the function, but I am not sure what {'Authorization': 'Bearer {}'.format(access_token)} does.

In [10]:
minf_album_id = '3tuFrjiGVuKMYiX7eY3s4P'
aboamo_album_id = '0SW2LQdzynFLJ05WzVYSJq'

Define the album id's.

In [11]:
#ab_tracks_ep.format(minf_album_id)

In [12]:
#ab_tracks_ep.format(aboamo_album_id)

Create api urls to the albums

In [13]:
minf_album_response = api_call(ab_tracks_ep.format(minf_album_id), session_header)
aboamo_album_response = api_call(ab_tracks_ep.format(aboamo_album_id), session_header)

200
200


These are making a call to the Spotify API with the full urls above.

In [14]:
minf_album_df = pd.DataFrame(minf_album_response['items'])
aboamo_album_df = pd.DataFrame(aboamo_album_response['items'])

Converts the data or "items" from the response of the API into tables.

In [15]:
#minf_album_df.head(11)

In [16]:
#aboamo_album_df.head(7)

Calls for the data tables, .head() shows the exact amount of tracks in the albums starting with 0.

In [17]:
minf_track_ids = ','.join(minf_album_df['id'].to_list())
aboamo_track_ids = ','.join(aboamo_album_df['id'].to_list())

Defines and retrieves the track ID's from the albums formed into a list.

In [18]:
#t_features_ep + '?ids={}'.format(minf_track_ids)

In [19]:
#t_features_ep + '?ids={}'.format(aboamo_track_ids)

Calls for the albums' track ID's to be put in a list.

In [20]:
minf_track_features = api_call(t_features_ep + '?ids={}'.format(minf_track_ids), session_header)
aboamo_track_features = api_call(t_features_ep + '?ids={}'.format(aboamo_track_ids), session_header)

200
200


Function to call for the track features and IDS

In [21]:
minf_track_info = api_call(track_ep + '?market=US&ids={}'.format(minf_track_ids), session_header)
aboamo_track_info = api_call(track_ep + '?market=US&ids={}'.format(aboamo_track_ids), session_header)

200
200


Function to call for where the songs are available in the market and the IDS

In [22]:
minf_features_df = pd.DataFrame(minf_track_features['audio_features'])
aboamo_features_df = pd.DataFrame(aboamo_track_features['audio_features'])

Function to call the audio features of each track from each album and put them in a table.

In [23]:
#minf_features_df.head(11)

In [24]:
#aboamo_features_df.head(7)

Calls the audio features for both albums.

In [25]:
minf_track_df = pd.DataFrame(minf_track_info['tracks'])
aboamo_track_df = pd.DataFrame(aboamo_track_info['tracks'])

Function that calls the information about each track from each album

In [26]:
#minf_track_df.head(11)

In [27]:
#aboamo_track_df.head(7)

Calls the funciton to provide all tracks and all information.

In [28]:
minf_merged = pd.merge(minf_features_df, minf_track_df, how = 'inner', on = 'id')

Merges track features and tracks for "May It Never Falter." I am not sure what how = 'inner', on = 'id' means.

In [29]:
#minf_merged.head(11)

Calls for the merged audio features and tracks in a table.

In [30]:
aboamo_merged = pd.merge(aboamo_features_df, aboamo_track_df, how = 'inner', on = 'id')

Merges track features and tracks for "a bit of a mad one" I am not sure what how = 'inner', on = 'id' means.

In [31]:
#aboamo_merged.head(7)

Calls for the merged audio features and tracks in a table.

In [32]:
minf_merged['EP'] = "May It Never Falter"
aboamo_merged['EP'] = "a bit of a mad one"

Defining that minf_merged is the EP "May It Never Falter" and that aboamo_merged is the "a bit of a mad one" EP.

In [33]:
minf_merged.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_x', 'id', 'uri_x', 'track_href', 'analysis_url', 'duration_ms_x',
       'time_signature', 'album', 'artists', 'disc_number', 'duration_ms_y',
       'explicit', 'external_ids', 'external_urls', 'href', 'is_local',
       'is_playable', 'name', 'popularity', 'preview_url', 'track_number',
       'type_y', 'uri_y', 'EP'],
      dtype='object')

Shows all of the possible columns.

In [34]:
minf = minf_merged.drop(columns = ['danceability', 'energy', 'key', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_x', 'id', 'uri_x', 'track_href', 'analysis_url', 'duration_ms_x',
       'time_signature', 'album', 'artists', 'disc_number', 'duration_ms_y',
       'explicit', 'external_ids', 'external_urls', 'href', 'is_local',
       'is_playable', 'popularity', 'preview_url', 'track_number',
       'type_y', 'uri_y', 'EP'])

Drops the selected colums.

In [35]:
#minf

Calls for the remaining columns.

In [36]:
minf['name'].iloc[[9]]

9    Nobodys Fault / Accept My Own
Name: name, dtype: object

Selects the name of the 9th most column.

In [37]:
aboamo_merged.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_x', 'id', 'uri_x', 'track_href', 'analysis_url', 'duration_ms_x',
       'time_signature', 'album', 'artists', 'disc_number', 'duration_ms_y',
       'explicit', 'external_ids', 'external_urls', 'href', 'is_local',
       'is_playable', 'name', 'popularity', 'track_number', 'type_y', 'uri_y',
       'EP'],
      dtype='object')

In [38]:
aboamo = aboamo_merged.drop(columns = ['danceability', 'energy', 'key', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_x', 'id', 'uri_x', 'track_href', 'analysis_url', 'duration_ms_x',
       'time_signature', 'album', 'artists', 'disc_number', 'duration_ms_y',
       'explicit', 'external_ids', 'external_urls', 'href', 'is_local',
       'is_playable', 'popularity', 'track_number', 'type_y', 'uri_y',
       'EP'])

In [39]:
#aboamo

In [40]:
aboamo['name'].iloc[[4]]

4    god is dead
Name: name, dtype: object

In [41]:
song_compare = pd.concat([minf[['loudness', 'name']].iloc[[9]], aboamo[['loudness', 'name']].iloc[[4]]], axis=0)

Merges minf and aboamo loudnesses and names of the 9th most column in minf and 4th most column in aboamo in a table.

In [42]:
#song_compare

Calls for the song_compare function.

In [43]:
song_compare_df = pd.DataFrame(song_compare[['loudness', 'name']])
song_compare_df.sort_values('loudness', ascending = False)

,loudness,name
9,-4.209,Nobodys Fault / Accept My Own
4,-6.304,god is dead


Compares the loudness of the songs from lowest to highest.

According to the table, it shows that "Nobodys Fault / Accept My Own" is louder than "god is dead" - "Nobodys Fault / Accept My Own" is closer to zero, therefore it is louder. According to my hypothesis, I hypothesized correctly by saying "god is dead" has a lower loudness value than "Nobodys Fault / Accept My Own." 

While doing this report, a limitation I came across was the "lack" of data Spotify provides. The loudness value, as well as other audio feature values, is a flat, generic number. I think that with this flat number, Spotify is lacking data that shows where parts of a song are loud and/or the different loudness values throughout a whole track- where is it the loudest, quietest? Another limitation is error. While I was completing this report, I had questioned where exactly these numbers came from and how they are calculated. I know that with loudness, it is calculated on a decible scale- but who or what is doing the calculation? Is there or has there been error with these calculations, misclick even? Though I do not know how some audio features are calculated such as valence and loudness, I think that an alternative to prevent error would be to calculate the audio feature multiple times and take the average, depending on how it is calculated.